In [1]:
import pandas as pd
import glob
import os
import openpyxl
from pathlib import Path
import xlwings as xw
import time

os.makedirs('エクセルファイル/投手データまとめ', exist_ok=True)

file_loadpath = 'エクセルファイル/チーム成績/*/*/投手成績/選手ごと/加工後/*/'

Ptypes = [['ストレート','L'],['ツーシーム','M'],['スライダー系','N'],['カーブ系','O'],['フォーク系','P'],['シンカー系','Q'],['シュート系','R']]



situations1 = [['得点圏',0],['対右',1],['対左',1]]
Ptypes2 = [['ストレート','L','U'],['ツーシーム','M','V'],['スライダー系','N','W'],['カーブ系','O','X'],['フォーク系','P','Y'],['シンカー系','Q','Z'],['シュート系','R','AA']]

situations2 = [['イニング',1,11],['カウント',2,21],['コース',2,27],['塁別',2,10],['月別',1,10],['曜日',1,8],['点差',2,11],['打順',1,10],['チーム別',0,12],]

situations3 = ['球場']


files = glob.glob('エクセルファイル/選手情報/*.xlsx')#読み込みたいファイル名
list = []
for file in files:
    list.append(pd.read_excel(file))
dfp = pd.concat(list).loc[:,'playerID':'Name_main']



In [2]:
def timecount(t1,t2):
    # 経過時間を表示
    print(f"経過時間：{t2-t1}")

def swco(swco):
    if type(swco) == float:
        swco *= 100
        return f'{swco:.02f}'
    else:
        return swco
    
def avg(avg):
    if type(avg) == float:
        return f'{avg:.03f}'
    else:
        return avg
    
def PAper(PAper):
    if type(PAper) == float:
        return f'{PAper:.02f}'
    else:
        return PAper
    
def pitper(pitper):
    if type(pitper) == float:
        pitper *= 100
        return f'{pitper:.01f}'
    else:
        return pitper    
    
def pid(Name_main):
    if Name_main != '坂本 勇人':
        try:
            ff = dfp[(dfp['Name_main'].str.contains(Name_main))].iloc[-1]
            playerID = ff['playerID']
            return playerID
        except:
            playerID = 999999
            return playerID
    else:
        playerID = 700003
        return playerID

def listin(Name_main,Stype,per,BC,O,hit,single,double,triple,homerun,TB,R,K,BB,DB,DP,OBA,WHIP,ERA,Sw,Co,SSw,SCo,BSw,BCo,GO,FO,RO,pitcount,PperPA,sitnum):
    ALL_Data = {'pID': pid(Name_main),
                'Name_main': Name_main,
                'Stype': Stype,
                'per': pitper(per),
                'BC': BC, #打者数
                'O': O, #奪アウト数
                'hit': hit,
                'single': single,
                'double': double,
                'triple': triple,
                'homerun': homerun,
                'TB': TB,
                'R': R,
                'K': K,
                'BB': BB,
                'DB': DB,
                'DP': DP,
                'OBA': avg(OBA),
                'WHIP': avg(WHIP),
                'ERA': avg(ERA),
                'Sw': swco(Sw),
                'Co': swco(Co),
                'SSw': swco(SSw),
                'SCo': swco(SCo),
                'BSw': swco(BSw),
                'BCo': swco(BCo),
                'GO': GO,
                'FO': FO,
                'RO': RO,
                'pitcount': pitcount,
                'PperPA': PAper(PperPA),
                'sitnum': sitnum,
                }
    
    ALL_Datas.append(ALL_Data)
    
def excelin(filename):
    pd.DataFrame(ALL_Datas).to_excel(f'エクセルファイル/投手データまとめ/{filename}.xlsx', index = False)

In [100]:
xw.App(visible=False)
t1 = time.time()#処理前の時刻
ALL_Datas = []
sitnum = 0
files = glob.glob(f"{file_loadpath}*全球*.xlsx")#読み込みたいファイル名
for file in files:
    wb = xw.Book(file)
    sht = wb.sheets[1]
    
    #全球、基本
    Name_main = Path(file).parts[-2]
    Stype = '全球'
    BC = sht.range("F3").value
    O = sht.range("C3").value
    hit = sht.range("G3").value
    single = sht.range("H3").value
    double = sht.range("I3").value
    triple = sht.range("J3").value
    homerun = sht.range("K3").value
    TB = int(single)+int(double)*2+int(triple)*3+int(homerun)*4
    R = sht.range("N3").value
    K = sht.range("B3").value
    BB = sht.range("L3").value
    DB = sht.range("M4").value
    DP = sht.range("O3").value
    OBA = sht.range("D7").value
    WHIP = sht.range("A7").value
    ERA = sht.range("A3").value
    Sw = sht.range("A11").value
    Co = sht.range("B11").value
    SSw = sht.range("D11").value
    SCo = sht.range("E11").value
    BSw = sht.range("G11").value
    BCo = sht.range("H11").value
    GO = sht.range("A15").value
    FO = sht.range("B15").value
    RO = sht.range("C15").value

    per = 100
    pitcount = sht.range("E3").value
    PperPA = sht.range("P7").value
    
    listin(Name_main,Stype,per,BC,O,hit,single,double,triple,homerun,TB,R,K,BB,DB,DP,OBA,WHIP,ERA,Sw,Co,SSw,SCo,BSw,BCo,GO,FO,RO,pitcount,PperPA,sitnum)
    
    #全球、球種別
    for item in Ptypes:
        Stype = item[0]
        BC = ''
        O = ''
        single = sht.range(f"{item[1]}33").value
        double = sht.range(f"{item[1]}34").value
        triple = sht.range(f"{item[1]}35").value
        homerun = sht.range(f"{item[1]}36").value
        hit = int(single)+int(double)+int(triple)+int(homerun)
        TB = int(single)+int(double)*2+int(triple)*3+int(homerun)*4
        R = ''
        K = sht.range(f"{item[1]}39").value
        BB = ''
        DB = ''
        DP = ''
        OBA = sht.range(f"{item[1]}32").value
        WHIP = ''
        ERA = ''

        GO = sht.range(f"{item[1]}41").value
        FO = sht.range(f"{item[1]}42").value
        RO = sht.range(f"{item[1]}43").value
        
        pitcount = sht.range(f"{item[1]}13").value
        
        Sw = sht.range(f"{item[1]}18").value
        Co = sht.range(f"{item[1]}19").value
        SSw = sht.range(f"{item[1]}24").value
        SCo = sht.range(f"{item[1]}25").value
        BSw = sht.range(f"{item[1]}30").value
        BCo = sht.range(f"{item[1]}31").value
        per = sht.range(f"{item[1]}14").value
        PperBC = sht.range(f"{item[1]}13").value/sht.range("F3").value
        
        listin(Name_main,Stype,per,BC,O,hit,single,double,triple,homerun,TB,R,K,BB,DB,DP,OBA,WHIP,ERA,Sw,Co,SSw,SCo,BSw,BCo,GO,FO,RO,pitcount,PperPA,sitnum)
        
    wb.close()
    wb = None


excelin('全球')
sitnum += 1
t2 = time.time()
timecount(t1,t2)

#シチュエーション1群
for situ1 in situations1:
    t1 = time.time()#処理前の時刻
    ALL_Datas = []
    files = glob.glob(f"{file_loadpath}*{situ1[0]}*.xlsx")
    for file in files:
        wb = xw.Book(file)
        sht = wb.sheets[situ1[1]]
        
        #シチュエーション１群、基本
        Name_main = Path(file).parts[-2]
        Stype = '全球'
        BC = sht.range("B2").value
        O = ''
        hit = sht.range("C2").value
        single = sht.range("D2").value
        double = sht.range("E2").value
        triple = sht.range("F2").value
        homerun = sht.range("G2").value
        TB = int(single)+int(double)*2+int(triple)*3+int(homerun)*4
        R = sht.range("K2").value
        K = sht.range("H2").value
        BB = sht.range("I2").value
        DB = sht.range("J2").value
        DP = ''
        OBA = sht.range("A5").value
        WHIP = ''
        ERA = ''
        Sw = sht.range("A8").value
        Co = sht.range("B8").value
        SSw = sht.range("D8").value
        SCo = sht.range("E8").value
        BSw = sht.range("G8").value
        BCo = sht.range("H8").value
        GO = sht.range("A12").value
        FO = sht.range("B12").value
        RO = sht.range("C12").value

        per = 100
        pitcount = sht.range("AE1").value

        try:
            PperPA = sht.range("AE1").value/sht.range("B2").value
        except ZeroDivisionError:
            PperPA = 0
        
        listin(Name_main,Stype,per,BC,O,hit,single,double,triple,homerun,TB,R,K,BB,DB,DP,OBA,WHIP,ERA,Sw,Co,SSw,SCo,BSw,BCo,GO,FO,RO,pitcount,PperPA,sitnum)

        #シチュエーション1群、球種別
        for item in Ptypes2:
            Name_main = Path(file).parts[-2]
            Stype = item[0]
            BC = ''
            O = ''
            single = sht.range(f"{item[2]}7").value
            double = sht.range(f"{item[2]}8").value
            triple = sht.range(f"{item[2]}9").value
            homerun = sht.range(f"{item[2]}10").value
            hit = int(single)+int(double)+int(triple)+int(homerun)
            TB = int(single)+int(double)*2+int(triple)*3+int(homerun)*4
            R = ''
            K = sht.range(f"{item[2]}13").value
            BB = ''
            DB = ''
            DP = ''
            OBA = sht.range(f"{item[2]}6").value
            WHIP = ''
            ERA = ''
            Sw = sht.range(f"{item[1]}11").value
            Co = sht.range(f"{item[1]}12").value
            SSw = sht.range(f"{item[1]}17").value
            SCo = sht.range(f"{item[1]}18").value
            BSw = sht.range(f"{item[1]}23").value
            BCo = sht.range(f"{item[1]}24").value
            GO = sht.range(f"{item[2]}15").value
            FO = sht.range(f"{item[2]}16").value
            RO = sht.range(f"{item[2]}17").value

            per = sht.range(f"{item[1]}7").value
            pitcount = sht.range(f"{item[1]}6").value
            try:
                PperPA = sht.range(f"{item[1]}6").value/sht.range("B2").value
            except ZeroDivisionError:
                PperPA = 0
            
            listin(Name_main,Stype,per,BC,O,hit,single,double,triple,homerun,TB,R,K,BB,DB,DP,OBA,WHIP,ERA,Sw,Co,SSw,SCo,BSw,BCo,GO,FO,RO,pitcount,PperPA,sitnum)
        
        wb.close()
        wb = None

        
    excelin(situ1[0])
    sitnum += 1
    t2 = time.time()
    timecount(t1,t2)
    
#シチュエーション2群
for situ2 in situations2:
    t1 = time.time()#処理前の時刻
    ALL_Datas = []
    files = glob.glob(f"{file_loadpath}*{situ2[0]}*.xlsx")
    for file in files:
        wb = xw.Book(file)
        for num in range(situ2[1],situ2[2]):
            sht = wb.sheets[num]
            
            #状況別、基本
            Name_main = Path(file).parts[-2]
            Stype = sht.name
            BC = sht.range("B2").value
            O = ''
            hit = sht.range("C2").value
            single = sht.range("D2").value
            double = sht.range("E2").value
            triple = sht.range("F2").value
            homerun = sht.range("G2").value
            TB = int(single)+int(double)*2+int(triple)*3+int(homerun)*4
            R = sht.range("K2").value
            K = sht.range("H2").value
            BB = sht.range("I2").value
            DB = sht.range("J2").value
            DP = ''
            OBA = sht.range("A5").value
            WHIP = ''
            ERA = ''
            Sw = sht.range("A8").value
            Co = sht.range("B8").value
            SSw = sht.range("D8").value
            SCo = sht.range("E8").value
            BSw = sht.range("G8").value
            BCo = sht.range("H8").value
            GO = sht.range("A12").value
            FO = sht.range("B12").value
            RO = sht.range("C12").value

            per = ''
            pitcount = sht.range("AE1").value

            try:
                PperPA = sht.range("AE1").value/sht.range("B2").value
            except ZeroDivisionError:
                PperPA = 0

            listin(Name_main,Stype,per,BC,O,hit,single,double,triple,homerun,TB,R,K,BB,DB,DP,OBA,WHIP,ERA,Sw,Co,SSw,SCo,BSw,BCo,GO,FO,RO,pitcount,PperPA,sitnum)

            
        wb.close()
        wb = None

            
    excelin(situ2[0])
    sitnum += 1
    t2 = time.time()
    timecount(t1,t2)
            
#シチュエーション3群
for situ3 in situations3:
    t1 = time.time()#処理前の時刻
    ALL_Datas = []
    files = glob.glob(f"{file_loadpath}*{situ3}*.xlsx")
    for file in files:
        wb = xw.Book(file)
        shts = wb.sheets
        for sn in range(0,len(shts)):
            sht = wb.sheets[sn]
            
            #状況別、基本
            Name_main = Path(file).parts[-2]
            Stype = shts[sn].name
            BC = sht.range("B2").value
            O = ''
            hit = sht.range("C2").value
            single = sht.range("D2").value
            double = sht.range("E2").value
            triple = sht.range("F2").value
            homerun = sht.range("G2").value
            TB = int(single)+int(double)*2+int(triple)*3+int(homerun)*4
            R = sht.range("K2").value
            K = sht.range("H2").value
            BB = sht.range("I2").value
            DB = sht.range("J2").value
            DP = ''
            OBA = sht.range("A5").value
            WHIP = ''
            ERA = ''
            Sw = sht.range("A8").value
            Co = sht.range("B8").value
            SSw = sht.range("D8").value
            SCo = sht.range("E8").value
            BSw = sht.range("G8").value
            BCo = sht.range("H8").value
            GO = sht.range("A12").value
            FO = sht.range("B12").value
            RO = sht.range("C12").value

            per = ''
            pitcount = sht.range("AE1").value

            try:
                PperPA = sht.range("AE1").value/sht.range("B2").value
            except ZeroDivisionError:
                PperPA = 0

            listin(Name_main,Stype,per,BC,O,hit,single,double,triple,homerun,TB,R,K,BB,DB,DP,OBA,WHIP,ERA,Sw,Co,SSw,SCo,BSw,BCo,GO,FO,RO,pitcount,PperPA,sitnum)

            
        wb.close()
        wb = None

        
    excelin(situ3)
    sitnum += 1
    t2 = time.time()
    timecount(t1,t2)
    
print('終了')

経過時間：272.40284299850464
経過時間：774.477213382721
経過時間：585.1965906620026
経過時間：540.7827517986298
経過時間：532.3240671157837
経過時間：983.1457612514496
経過時間：1085.5679817199707
経過時間：420.0928461551666
経過時間：456.8178074359894
経過時間：377.3162467479706
経過時間：463.68721532821655
経過時間：456.1665313243866
経過時間：531.9929797649384
経過時間：355.41267347335815


In [3]:
import pandas as pd
import glob
import os
import openpyxl
from pathlib import Path
import xlwings as xw
import time


os.makedirs('エクセルファイル/各球団投手データまとめ', exist_ok=True)

file_loadpath = 'エクセルファイル/チーム成績/*/*/投手成績/チーム全体/加工後/'

situations2 = [['イニング',1,11],['カウント',2,21],['コース',2,27],['塁別',2,10],['月別',1,10],['曜日',1,8],['点差',2,11],['打順',1,10],['チーム別',0,12],['守備位置別',1,12]]


TN = [['巨人','10'],
      ['阪神','11'],
      ['中日','12'],
      ['DeNA','13'],
      ['広島','14'],
      ['ヤクルト','15'],
      ['ソフトバンク','20'],
      ['ロッテ','21'],
      ['西武','22'],
      ['楽天','23'],
      ['日本ハム','24'],
      ['オリックス','25'],
     ]

def tid(tName):
    for t in TN:
        if tName in t[0]:
            Team_ID = t[1]
            return Team_ID

def listin(tName,Stype,per,BC,O,hit,single,double,triple,homerun,TB,R,K,BB,DB,DP,OBA,WHIP,ERA,Sw,Co,SSw,SCo,BSw,BCo,GO,FO,RO,pitcount,PperPA,sitnum):
    ALL_Data = {'tID': tid(tName),
                'tName': tName,
                'Stype': Stype,
                'per': pitper(per),
                'BC': BC, #打者数
                'O': O, #奪アウト数
                'hit': hit,
                'single': single,
                'double': double,
                'triple': triple,
                'homerun': homerun,
                'TB': TB,
                'R': R,
                'K': K,
                'BB': BB,
                'DB': DB,
                'DP': DP,
                'OBA': avg(OBA),
                'WHIP': avg(WHIP),
                'ERA': avg(ERA),
                'Sw': swco(Sw),
                'Co': swco(Co),
                'SSw': swco(SSw),
                'SCo': swco(SCo),
                'BSw': swco(BSw),
                'BCo': swco(BCo),
                'GO': GO,
                'FO': FO,
                'RO': RO,
                'pitcount': pitcount,
                'PperPA': PAper(PperPA),
                'sitnum': sitnum,
                }
    
    ALL_Datas.append(ALL_Data)
    
def excelin(filename):
    pd.DataFrame(ALL_Datas).to_excel(f'エクセルファイル/各球団投手データまとめ/{filename}.xlsx', index = False)

In [ ]:
xw.App(visible=False)
t1 = time.time()#処理前の時刻
ALL_Datas = []
sitnum = 0
files = glob.glob(f"{file_loadpath}*全球*.xlsx")#読み込みたいファイル名
for file in files:
    wb = xw.Book(file)
    sht = wb.sheets[1]
    
    #全球、基本
    tName = Path(file).parts[-5]
    Stype = '全球'
    BC = sht.range("F3").value
    O = sht.range("C3").value
    hit = sht.range("G3").value
    single = sht.range("H3").value
    double = sht.range("I3").value
    triple = sht.range("J3").value
    homerun = sht.range("K3").value
    TB = int(single)+int(double)*2+int(triple)*3+int(homerun)*4
    R = sht.range("N3").value
    K = sht.range("B3").value
    BB = sht.range("L3").value
    DB = sht.range("M4").value
    DP = sht.range("O3").value
    OBA = sht.range("D7").value
    WHIP = sht.range("A7").value
    ERA = sht.range("A3").value
    Sw = sht.range("A11").value
    Co = sht.range("B11").value
    SSw = sht.range("D11").value
    SCo = sht.range("E11").value
    BSw = sht.range("G11").value
    BCo = sht.range("H11").value
    GO = sht.range("A15").value
    FO = sht.range("B15").value
    RO = sht.range("C15").value

    per = 100
    pitcount = sht.range("E3").value
    PperPA = sht.range("P7").value
    
    listin(tName,Stype,per,BC,O,hit,single,double,triple,homerun,TB,R,K,BB,DB,DP,OBA,WHIP,ERA,Sw,Co,SSw,SCo,BSw,BCo,GO,FO,RO,pitcount,PperPA,sitnum):
    
    #全球、球種別
    for item in Ptypes:
        Stype = item[0]
        BC = ''
        O = ''
        single = sht.range(f"{item[1]}33").value
        double = sht.range(f"{item[1]}34").value
        triple = sht.range(f"{item[1]}35").value
        homerun = sht.range(f"{item[1]}36").value
        hit = int(single)+int(double)+int(triple)+int(homerun)
        TB = int(single)+int(double)*2+int(triple)*3+int(homerun)*4
        R = ''
        K = sht.range(f"{item[1]}39").value
        BB = ''
        DB = ''
        DP = ''
        OBA = sht.range(f"{item[1]}32").value
        WHIP = ''
        ERA = ''

        GO = sht.range(f"{item[1]}41").value
        FO = sht.range(f"{item[1]}42").value
        RO = sht.range(f"{item[1]}43").value
        
        pitcount = sht.range(f"{item[1]}13").value
        
        Sw = sht.range(f"{item[1]}18").value
        Co = sht.range(f"{item[1]}19").value
        SSw = sht.range(f"{item[1]}24").value
        SCo = sht.range(f"{item[1]}25").value
        BSw = sht.range(f"{item[1]}30").value
        BCo = sht.range(f"{item[1]}31").value
        per = sht.range(f"{item[1]}14").value
        PperBC = sht.range(f"{item[1]}13").value/sht.range("F3").value
        
        listin(tName,Stype,per,BC,O,hit,single,double,triple,homerun,TB,R,K,BB,DB,DP,OBA,WHIP,ERA,Sw,Co,SSw,SCo,BSw,BCo,GO,FO,RO,pitcount,PperPA,sitnum)
        
    wb.close()
    wb = None


excelin('全球')
sitnum += 1
t2 = time.time()
timecount(t1,t2)

#シチュエーション1群
for situ1 in situations1:
    t1 = time.time()#処理前の時刻
    ALL_Datas = []
    files = glob.glob(f"{file_loadpath}*{situ1[0]}*.xlsx")
    for file in files:
        wb = xw.Book(file)
        sht = wb.sheets[situ1[1]]
        
        #シチュエーション１群、基本
        tName = Path(file).parts[-5]
        Stype = '全球'
        BC = sht.range("B2").value
        O = ''
        hit = sht.range("C2").value
        single = sht.range("D2").value
        double = sht.range("E2").value
        triple = sht.range("F2").value
        homerun = sht.range("G2").value
        TB = int(single)+int(double)*2+int(triple)*3+int(homerun)*4
        R = sht.range("K2").value
        K = sht.range("H2").value
        BB = sht.range("I2").value
        DB = sht.range("J2").value
        DP = ''
        OBA = sht.range("A5").value
        WHIP = ''
        ERA = ''
        Sw = sht.range("A8").value
        Co = sht.range("B8").value
        SSw = sht.range("D8").value
        SCo = sht.range("E8").value
        BSw = sht.range("G8").value
        BCo = sht.range("H8").value
        GO = sht.range("A12").value
        FO = sht.range("B12").value
        RO = sht.range("C12").value

        per = 100
        pitcount = sht.range("AE1").value

        try:
            PperPA = sht.range("AE1").value/sht.range("B2").value
        except ZeroDivisionError:
            PperPA = 0
            
        listin(tName,Stype,per,BC,O,hit,single,double,triple,homerun,TB,R,K,BB,DB,DP,OBA,WHIP,ERA,Sw,Co,SSw,SCo,BSw,BCo,GO,FO,RO,pitcount,PperPA,sitnum)

        #シチュエーション1群、球種別
        for item in Ptypes2:
            tName = Path(file).parts[-5]
            Stype = item[0]
            BC = ''
            O = ''
            single = sht.range(f"{item[2]}7").value
            double = sht.range(f"{item[2]}8").value
            triple = sht.range(f"{item[2]}9").value
            homerun = sht.range(f"{item[2]}10").value
            hit = int(single)+int(double)+int(triple)+int(homerun)
            TB = int(single)+int(double)*2+int(triple)*3+int(homerun)*4
            R = ''
            K = sht.range(f"{item[2]}13").value
            BB = ''
            DB = ''
            DP = ''
            OBA = sht.range(f"{item[2]}6").value
            WHIP = ''
            ERA = ''
            Sw = sht.range(f"{item[1]}11").value
            Co = sht.range(f"{item[1]}12").value
            SSw = sht.range(f"{item[1]}17").value
            SCo = sht.range(f"{item[1]}18").value
            BSw = sht.range(f"{item[1]}23").value
            BCo = sht.range(f"{item[1]}24").value
            GO = sht.range(f"{item[2]}15").value
            FO = sht.range(f"{item[2]}16").value
            RO = sht.range(f"{item[2]}17").value

            per = sht.range(f"{item[1]}7").value
            pitcount = sht.range(f"{item[1]}6").value
            try:
                PperPA = sht.range(f"{item[1]}6").value/sht.range("B2").value
            except ZeroDivisionError:
                PperPA = 0
            
            listin(tName,Stype,per,BC,O,hit,single,double,triple,homerun,TB,R,K,BB,DB,DP,OBA,WHIP,ERA,Sw,Co,SSw,SCo,BSw,BCo,GO,FO,RO,pitcount,PperPA,sitnum)
        
        wb.close()
        wb = None

        
    excelin(situ1[0])
    sitnum += 1
    t2 = time.time()
    timecount(t1,t2)
    
#シチュエーション2群
for situ2 in situations2:
    t1 = time.time()#処理前の時刻
    ALL_Datas = []
    files = glob.glob(f"{file_loadpath}*{situ2[0]}*.xlsx")
    for file in files:
        wb = xw.Book(file)
        for num in range(situ2[1],situ2[2]):
            sht = wb.sheets[num]
            
            #状況別、基本
            tName = Path(file).parts[-5]
            Stype = sht.name
            BC = sht.range("B2").value
            O = ''
            hit = sht.range("C2").value
            single = sht.range("D2").value
            double = sht.range("E2").value
            triple = sht.range("F2").value
            homerun = sht.range("G2").value
            TB = int(single)+int(double)*2+int(triple)*3+int(homerun)*4
            R = sht.range("K2").value
            K = sht.range("H2").value
            BB = sht.range("I2").value
            DB = sht.range("J2").value
            DP = ''
            OBA = sht.range("A5").value
            WHIP = ''
            ERA = ''
            Sw = sht.range("A8").value
            Co = sht.range("B8").value
            SSw = sht.range("D8").value
            SCo = sht.range("E8").value
            BSw = sht.range("G8").value
            BCo = sht.range("H8").value
            GO = sht.range("A12").value
            FO = sht.range("B12").value
            RO = sht.range("C12").value

            per = ''
            pitcount = sht.range("AE1").value

            try:
                PperPA = sht.range("AE1").value/sht.range("B2").value
            except ZeroDivisionError:
                PperPA = 0

            listin(tName,Stype,per,BC,O,hit,single,double,triple,homerun,TB,R,K,BB,DB,DP,OBA,WHIP,ERA,Sw,Co,SSw,SCo,BSw,BCo,GO,FO,RO,pitcount,PperPA,sitnum)
            
        wb.close()
        wb = None

            
    excelin(situ2[0])
    sitnum += 1
    t2 = time.time()
    timecount(t1,t2)
            
#シチュエーション3群
for situ3 in situations3:
    t1 = time.time()#処理前の時刻
    ALL_Datas = []
    files = glob.glob(f"{file_loadpath}*{situ3}*.xlsx")
    for file in files:
        wb = xw.Book(file)
        shts = wb.sheets
        for sn in range(0,len(shts)):
            sht = wb.sheets[sn]
            
            #状況別、基本
            tName = Path(file).parts[-5]
            Stype = shts[sn].name
            BC = sht.range("B2").value
            O = ''
            hit = sht.range("C2").value
            single = sht.range("D2").value
            double = sht.range("E2").value
            triple = sht.range("F2").value
            homerun = sht.range("G2").value
            TB = int(single)+int(double)*2+int(triple)*3+int(homerun)*4
            R = sht.range("K2").value
            K = sht.range("H2").value
            BB = sht.range("I2").value
            DB = sht.range("J2").value
            DP = ''
            OBA = sht.range("A5").value
            WHIP = ''
            ERA = ''
            Sw = sht.range("A8").value
            Co = sht.range("B8").value
            SSw = sht.range("D8").value
            SCo = sht.range("E8").value
            BSw = sht.range("G8").value
            BCo = sht.range("H8").value
            GO = sht.range("A12").value
            FO = sht.range("B12").value
            RO = sht.range("C12").value

            per = ''
            pitcount = sht.range("AE1").value

            try:
                PperPA = sht.range("AE1").value/sht.range("B2").value
            except ZeroDivisionError:
                PperPA = 0

                
            listin(tName,Stype,per,BC,O,hit,single,double,triple,homerun,TB,R,K,BB,DB,DP,OBA,WHIP,ERA,Sw,Co,SSw,SCo,BSw,BCo,GO,FO,RO,pitcount,PperPA,sitnum)

            
        wb.close()
        wb = None

        
    excelin(situ3)
    sitnum += 1
    t2 = time.time()
    timecount(t1,t2)
    
print('終了')

In [ ]:
os.makedirs('エクセルファイル/各リーグ投手データまとめ', exist_ok=True)

file_loadpath = 'エクセルファイル/チーム成績/*/*/投手成績/加工後/'



LN = [['セリーグ全体', '1' ,'セリーグ'],
      ['パリーグ全体', '2' ,'パリーグ'],
      ['両リーグ全体', '3' ,'両リーグ'],
     ]

def lid(lName):
    for l in LN:
        if l[0] in lName:
            league_ID = l[1]
            return league_ID
        
def league(lName):
    for l in LN:
        if l[0] in lName:
            leagueN = l[2]
            return leagueN

def listin(lName,Stype,per,BC,O,hit,single,double,triple,homerun,TB,R,K,BB,DB,DP,OBA,WHIP,ERA,Sw,Co,SSw,SCo,BSw,BCo,GO,FO,RO,pitcount,PperPA,sitnum):
    ALL_Data = {'LID': lid(lName),
                'lName': league(lName),
                'Stype': Stype,
                'per': pitper(per),
                'BC': BC, #打者数
                'O': O, #奪アウト数
                'hit': hit,
                'single': single,
                'double': double,
                'triple': triple,
                'homerun': homerun,
                'TB': TB,
                'R': R,
                'K': K,
                'BB': BB,
                'DB': DB,
                'DP': DP,
                'OBA': avg(OBA),
                'WHIP': avg(WHIP),
                'ERA': avg(ERA),
                'Sw': swco(Sw),
                'Co': swco(Co),
                'SSw': swco(SSw),
                'SCo': swco(SCo),
                'BSw': swco(BSw),
                'BCo': swco(BCo),
                'GO': GO,
                'FO': FO,
                'RO': RO,
                'pitcount': pitcount,
                'PperPA': PAper(PperPA),
                'sitnum': sitnum,
                }
    
    ALL_Datas.append(ALL_Data)
    

def excelin(filename):
    pd.DataFrame(ALL_Datas).to_excel(f'エクセルファイル/各リーグ投手データまとめ/{filename}.xlsx', index = False)

In [ ]:
xw.App(visible=False)
t1 = time.time()#処理前の時刻
ALL_Datas = []
sitnum = 0
files = glob.glob(f"{file_loadpath}*全球*.xlsx")#読み込みたいファイル名
for file in files:
    wb = xw.Book(file)
    sht = wb.sheets[1]
    
    #全球、基本
    lName = Path(file).parts[-4]
    Stype = '全球'
    BC = sht.range("F3").value
    O = sht.range("C3").value
    hit = sht.range("G3").value
    single = sht.range("H3").value
    double = sht.range("I3").value
    triple = sht.range("J3").value
    homerun = sht.range("K3").value
    TB = int(single)+int(double)*2+int(triple)*3+int(homerun)*4
    R = sht.range("N3").value
    K = sht.range("B3").value
    BB = sht.range("L3").value
    DB = sht.range("M4").value
    DP = sht.range("O3").value
    OBA = sht.range("D7").value
    WHIP = sht.range("A7").value
    ERA = sht.range("A3").value
    Sw = sht.range("A11").value
    Co = sht.range("B11").value
    SSw = sht.range("D11").value
    SCo = sht.range("E11").value
    BSw = sht.range("G11").value
    BCo = sht.range("H11").value
    GO = sht.range("A15").value
    FO = sht.range("B15").value
    RO = sht.range("C15").value

    per = 100
    pitcount = sht.range("E3").value
    PperPA = sht.range("P7").value
    
    listin(lName,Stype,per,BC,O,hit,single,double,triple,homerun,TB,R,K,BB,DB,DP,OBA,WHIP,ERA,Sw,Co,SSw,SCo,BSw,BCo,GO,FO,RO,pitcount,PperPA,sitnum):
    
    #全球、球種別
    for item in Ptypes:
        Stype = item[0]
        BC = ''
        O = ''
        single = sht.range(f"{item[1]}33").value
        double = sht.range(f"{item[1]}34").value
        triple = sht.range(f"{item[1]}35").value
        homerun = sht.range(f"{item[1]}36").value
        hit = int(single)+int(double)+int(triple)+int(homerun)
        TB = int(single)+int(double)*2+int(triple)*3+int(homerun)*4
        R = ''
        K = sht.range(f"{item[1]}39").value
        BB = ''
        DB = ''
        DP = ''
        OBA = sht.range(f"{item[1]}32").value
        WHIP = ''
        ERA = ''

        GO = sht.range(f"{item[1]}41").value
        FO = sht.range(f"{item[1]}42").value
        RO = sht.range(f"{item[1]}43").value
        
        pitcount = sht.range(f"{item[1]}13").value
        
        Sw = sht.range(f"{item[1]}18").value
        Co = sht.range(f"{item[1]}19").value
        SSw = sht.range(f"{item[1]}24").value
        SCo = sht.range(f"{item[1]}25").value
        BSw = sht.range(f"{item[1]}30").value
        BCo = sht.range(f"{item[1]}31").value
        per = sht.range(f"{item[1]}14").value
        PperBC = sht.range(f"{item[1]}13").value/sht.range("F3").value
        
        listin(lName,Stype,per,BC,O,hit,single,double,triple,homerun,TB,R,K,BB,DB,DP,OBA,WHIP,ERA,Sw,Co,SSw,SCo,BSw,BCo,GO,FO,RO,pitcount,PperPA,sitnum)
        
    wb.close()
    wb = None


excelin('全球')
sitnum += 1
t2 = time.time()
timecount(t1,t2)

#シチュエーション1群
for situ1 in situations1:
    t1 = time.time()#処理前の時刻
    ALL_Datas = []
    files = glob.glob(f"{file_loadpath}*{situ1[0]}*.xlsx")
    for file in files:
        wb = xw.Book(file)
        sht = wb.sheets[situ1[1]]
        
        #シチュエーション１群、基本
        lName = Path(file).parts[-4]
        Stype = '全球'
        BC = sht.range("B2").value
        O = ''
        hit = sht.range("C2").value
        single = sht.range("D2").value
        double = sht.range("E2").value
        triple = sht.range("F2").value
        homerun = sht.range("G2").value
        TB = int(single)+int(double)*2+int(triple)*3+int(homerun)*4
        R = sht.range("K2").value
        K = sht.range("H2").value
        BB = sht.range("I2").value
        DB = sht.range("J2").value
        DP = ''
        OBA = sht.range("A5").value
        WHIP = ''
        ERA = ''
        Sw = sht.range("A8").value
        Co = sht.range("B8").value
        SSw = sht.range("D8").value
        SCo = sht.range("E8").value
        BSw = sht.range("G8").value
        BCo = sht.range("H8").value
        GO = sht.range("A12").value
        FO = sht.range("B12").value
        RO = sht.range("C12").value

        per = 100
        pitcount = sht.range("AE1").value

        try:
            PperPA = sht.range("AE1").value/sht.range("B2").value
        except ZeroDivisionError:
            PperPA = 0
            
        listin(lName,Stype,per,BC,O,hit,single,double,triple,homerun,TB,R,K,BB,DB,DP,OBA,WHIP,ERA,Sw,Co,SSw,SCo,BSw,BCo,GO,FO,RO,pitcount,PperPA,sitnum)

        #シチュエーション1群、球種別
        for item in Ptypes2:
            lName = Path(file).parts[-4]
            Stype = item[0]
            BC = ''
            O = ''
            single = sht.range(f"{item[2]}7").value
            double = sht.range(f"{item[2]}8").value
            triple = sht.range(f"{item[2]}9").value
            homerun = sht.range(f"{item[2]}10").value
            hit = int(single)+int(double)+int(triple)+int(homerun)
            TB = int(single)+int(double)*2+int(triple)*3+int(homerun)*4
            R = ''
            K = sht.range(f"{item[2]}13").value
            BB = ''
            DB = ''
            DP = ''
            OBA = sht.range(f"{item[2]}6").value
            WHIP = ''
            ERA = ''
            Sw = sht.range(f"{item[1]}11").value
            Co = sht.range(f"{item[1]}12").value
            SSw = sht.range(f"{item[1]}17").value
            SCo = sht.range(f"{item[1]}18").value
            BSw = sht.range(f"{item[1]}23").value
            BCo = sht.range(f"{item[1]}24").value
            GO = sht.range(f"{item[2]}15").value
            FO = sht.range(f"{item[2]}16").value
            RO = sht.range(f"{item[2]}17").value

            per = sht.range(f"{item[1]}7").value
            pitcount = sht.range(f"{item[1]}6").value
            try:
                PperPA = sht.range(f"{item[1]}6").value/sht.range("B2").value
            except ZeroDivisionError:
                PperPA = 0
            
            listin(lName,Stype,per,BC,O,hit,single,double,triple,homerun,TB,R,K,BB,DB,DP,OBA,WHIP,ERA,Sw,Co,SSw,SCo,BSw,BCo,GO,FO,RO,pitcount,PperPA,sitnum)
        
        wb.close()
        wb = None

        
    excelin(situ1[0])
    sitnum += 1
    t2 = time.time()
    timecount(t1,t2)
    
#シチュエーション2群
for situ2 in situations2:
    t1 = time.time()#処理前の時刻
    ALL_Datas = []
    files = glob.glob(f"{file_loadpath}*{situ2[0]}*.xlsx")
    for file in files:
        wb = xw.Book(file)
        for num in range(situ2[1],situ2[2]):
            sht = wb.sheets[num]
            
            #状況別、基本
            lName = Path(file).parts[-4]
            Stype = sht.name
            BC = sht.range("B2").value
            O = ''
            hit = sht.range("C2").value
            single = sht.range("D2").value
            double = sht.range("E2").value
            triple = sht.range("F2").value
            homerun = sht.range("G2").value
            TB = int(single)+int(double)*2+int(triple)*3+int(homerun)*4
            R = sht.range("K2").value
            K = sht.range("H2").value
            BB = sht.range("I2").value
            DB = sht.range("J2").value
            DP = ''
            OBA = sht.range("A5").value
            WHIP = ''
            ERA = ''
            Sw = sht.range("A8").value
            Co = sht.range("B8").value
            SSw = sht.range("D8").value
            SCo = sht.range("E8").value
            BSw = sht.range("G8").value
            BCo = sht.range("H8").value
            GO = sht.range("A12").value
            FO = sht.range("B12").value
            RO = sht.range("C12").value

            per = ''
            pitcount = sht.range("AE1").value

            try:
                PperPA = sht.range("AE1").value/sht.range("B2").value
            except ZeroDivisionError:
                PperPA = 0

            listin(lName,Stype,per,BC,O,hit,single,double,triple,homerun,TB,R,K,BB,DB,DP,OBA,WHIP,ERA,Sw,Co,SSw,SCo,BSw,BCo,GO,FO,RO,pitcount,PperPA,sitnum)
            
        wb.close()
        wb = None

            
    excelin(situ2[0])
    sitnum += 1
    t2 = time.time()
    timecount(t1,t2)
            
#シチュエーション3群
for situ3 in situations3:
    t1 = time.time()#処理前の時刻
    ALL_Datas = []
    files = glob.glob(f"{file_loadpath}*{situ3}*.xlsx")
    for file in files:
        wb = xw.Book(file)
        shts = wb.sheets
        for sn in range(0,len(shts)):
            sht = wb.sheets[sn]
            
            #状況別、基本
            lName = Path(file).parts[-4]
            Stype = shts[sn].name
            BC = sht.range("B2").value
            O = ''
            hit = sht.range("C2").value
            single = sht.range("D2").value
            double = sht.range("E2").value
            triple = sht.range("F2").value
            homerun = sht.range("G2").value
            TB = int(single)+int(double)*2+int(triple)*3+int(homerun)*4
            R = sht.range("K2").value
            K = sht.range("H2").value
            BB = sht.range("I2").value
            DB = sht.range("J2").value
            DP = ''
            OBA = sht.range("A5").value
            WHIP = ''
            ERA = ''
            Sw = sht.range("A8").value
            Co = sht.range("B8").value
            SSw = sht.range("D8").value
            SCo = sht.range("E8").value
            BSw = sht.range("G8").value
            BCo = sht.range("H8").value
            GO = sht.range("A12").value
            FO = sht.range("B12").value
            RO = sht.range("C12").value

            per = ''
            pitcount = sht.range("AE1").value

            try:
                PperPA = sht.range("AE1").value/sht.range("B2").value
            except ZeroDivisionError:
                PperPA = 0

                
            listin(lName,Stype,per,BC,O,hit,single,double,triple,homerun,TB,R,K,BB,DB,DP,OBA,WHIP,ERA,Sw,Co,SSw,SCo,BSw,BCo,GO,FO,RO,pitcount,PperPA,sitnum)

            
        wb.close()
        wb = None

        
    excelin(situ3)
    sitnum += 1
    t2 = time.time()
    timecount(t1,t2)
    
print('終了')